## Indexes

In PostgreSQL, the CREATE INDEX statement is used to create an index on a table. 

Indexes improve the performance of queries by allowing the database to find rows more quickly, especially for large datasets.

They are particularly useful for speeding up SELECT queries with WHERE, JOIN, ORDER BY, or GROUP BY clauses.

Syntax:


In [ ]:
CREATE INDEX index_name ON table_name (column_name);

Key Points:
- Index Name: A unique name for the index.

- Table Name: The table on which the index is created.
- Column Name: The column(s) to be indexed.

Example:

In [ ]:
CREATE INDEX idx_employee_name ON employees (name);

This creates an index named idx_employee_name on the name column of the employees table, speeding up queries that filter or sort by name.

### Types of Indexes:
- `B-Tree` (default): Used for most queries (e.g., equality, range queries).

- `GIN` (Generalized Inverted Index): Used for full-text search or JSONB data.
- `GiST` (Generalized Search Tree): Used for geospatial data.
- `Hash`: Used for equality comparisons (e.g., =).
- `BRIN` (Block Range Index): Used for very large tables with sequential data.

Advanced Example:

In [ ]:
CREATE UNIQUE INDEX idx_employee_email ON employees (email);

This creates a unique index, ensuring that no two rows in the employees table have the same email.


-  In PostgreSQL, when you define a `UNIQUE` constraint on a column, PostgreSQL 
automatically creates a unique index on that column to enforce the constraint. Therefore, creating a unique index manually on a column that already has a `UNIQUE` constraint is redundant.



In [ ]:
-- Adding a UNIQUE constraint (automatically creates a unique index):
CREATE TABLE users (
    id SERIAL PRIMARY KEY,
    email VARCHAR(255) UNIQUE
);

-- Equivalent to:
CREATE TABLE users (
    id SERIAL PRIMARY KEY,
    email VARCHAR(255)
);
CREATE UNIQUE INDEX idx_users_email ON users(email);

### B-TREE INDEX

- A B-Tree index (short for Balanced Tree) is the default and most commonly used type of index in PostgreSQL. 

- It is designed to efficiently handle equality and range queries, making it suitable for a wide variety of use cases.

- Key Features of B-Tree Index:
    - Balanced Structure: The B-Tree is a self-balancing tree structure where all leaf nodes are at the same level.
    - This ensures that the time complexity for searching, inserting, or deleting is logarithmic `(O(log n))`.

- Ordered Data:
    - The B-Tree index stores data in a sorted order, which makes it ideal for queries involving:
        - Equality (`=`).
        - Range conditions (`<, <=, >, >=, BETWEEN`).
        - Sorting (`ORDER BY`).

- Multi-Column Index:
    - B-Tree indexes can be created on multiple columns, and they are useful when queries filter or sort by those columns.
- Default Index Type:
    - When you create an index without specifying the type, PostgreSQL uses a B-Tree index by default.
    
Syntax:

In [ ]:
CREATE INDEX index_name ON table_name (column_name);

In [ ]:
-- example:
CREATE INDEX idx_employee_salary ON employees (salary);

This creates a B-Tree index on the `salary` column of the `employees` table. 

Queries like the following will benefit from this index:

In [ ]:
SELECT * FROM employees WHERE salary > 50000;
SELECT * FROM employees ORDER BY salary;


-- Equality conditions:
SELECT * FROM employees WHERE department = 'HR';


-- Range queries:
SELECT * FROM employees WHERE salary BETWEEN 50000 AND 100000;


-- Sorting:
SELECT * FROM employees ORDER BY joining_date DESC;

- Limitations:
    - Not suitable for non-scalar data types like JSONB or ARRAY.
    
    - Not efficient for full-text search or geospatial queries (use GIN or GiST indexes instead).

--------------

### GIN (Generalized Inverted Index) and GiST (Generalized Search Index)

need to learn

----------------

### EXPLAIN AND EXPLAIN ANALYZE

In PostgreSQL, `EXPLAIN` and `EXPLAIN ANALYZE` are tools used to 
analyze query execution plans and identify performance bottlenecks. They provide insights into how the PostgreSQL query planner executes a query, including the use of indexes, join strategies, and estimated costs.

#### 1. EXPLAIN
- The `EXPLAIN` command shows the execution plan for a query without actually running it. It provides details about how PostgreSQL plans to execute the query, including the steps involved and their estimated costs.

Syntax:

In [ ]:
-- EXPLAIN query;

EXPLAIN SELECT * FROM employees WHERE salary > 50000;

Output:
- Seq Scan: Indicates a sequential scan of the table.
- Index Scan: Indicates the use of an index.
- Cost: Estimated cost of the operation (start-up cost and total cost).
- Rows: Estimated number of rows returned by the operation.


----------

#### 2. EXPLAIN ANALYZE

The `EXPLAIN ANALYZE` command executes the query and provides the actual execution plan along with runtime statistics. This is useful for comparing the estimated costs with the actual performance.

Syntax:


In [ ]:
-- EXPLAIN ANALYZE query; -- SYNTAX

EXPLAIN ANALYZE SELECT * FROM employees WHERE salary > 50000;

Output:
- Actual Time: The actual time taken for each step.
- Rows: The actual number of rows processed.
- Buffers: Information about disk I/O (if enabled).
- Timing: Total execution time.


Example with Index Optimization:

In [ ]:
-- Create an index on the salary column
CREATE INDEX idx_salary ON employees (salary);

-- Analyze the query
EXPLAIN ANALYZE SELECT * FROM employees WHERE salary > 50000;

If the index is used, the output will show an Index Scan instead of a Seq Scan, reducing the query execution time.

-------------------

## Performance Tuning


##### 1) Avoid SELECT * in production; fetch only required columns.

##### 2) Filter using indexed columns

##### 3) Avoid Functions on Indexed Columns

- Avoid applying functions or transformations to indexed columns in the `WHERE` clause, as it can prevent the index from being used. 
    
For example:

In [ ]:
-- Avoid: 

SELECT id, name 
FROM employees 
WHERE LOWER(department) = 'it';



-- Instead, use:

SELECT id, name 
FROM employees 
WHERE department = 'IT';

##### 4) Use Composite Indexes for Multiple Columns
If filtering involves multiple columns, create a composite index:



In [ ]:
CREATE INDEX idx_employees_dept_salary ON employees(department, salary);


-- Query example:

SELECT id, name 
FROM employees 
WHERE department = 'IT' AND salary > 50000;


-- By indexing and filtering effectively, you can significantly improve query performance.

##### 5) Reduce unnecessary subqueries and use Common Table Expressions (CTEs) for readability.


##### 6) Use pagination (`LIMIT`, `OFFSET`) efficiently for large datasets.

##### 7) Optimize joins by ensuring proper indexing on join keys.

##### 8) Monitor slow queries with `pg_stat_statements`

---------------

## Vacuuming and Autovacuum

In PostgreSQL, vacuuming is a maintenance operation that helps manage storage and optimize database performance by reclaiming unused space and updating table statistics. 

Over time, as rows are updated or deleted, PostgreSQL does not immediately remove the old data but marks it as "dead tuples." 

These dead tuples can accumulate and degrade performance if not cleaned up.

#### 1. Why Vacuuming is Needed
- PostgreSQL uses `MVCC (Multi-Version Concurrency Control)` to handle concurrent transactions. When rows are updated or deleted:

    - The old versions of rows (dead tuples) remain in the table until they are cleaned up.

    - Dead tuples consume disk space and can slow down queries.
- Vacuuming:
    - Reclaims storage occupied by dead tuples.

    - `Updates statistics for the query planner` to make better decisions.

    ----------
    -----

- ##### `Query Planner Statistics in PostgreSQL`

    - Query planner statistics are metadata collected by PostgreSQL about the data in your tables. These statistics help the query planner decide the most efficient way to execute a query.
    - The planner uses this information to estimate costs, choose indexes, determine join strategies, and optimize query execution.

1. Why Are `Query Planner Statistics` Important?
- PostgreSQL's query planner relies on accurate statistics to:
    - Estimate the number of rows a query will return.
    - Decide whether to use an index scan or a sequential scan.
    - Optimize joins, sorting, and filtering operations.
    - Minimize query execution time by selecting the best execution plan.

If the statistics are outdated or inaccurate, the query planner may make suboptimal decisions, leading to poor performance.

---------------


2. How Are `Statistics Collected`?
- Statistics are collected and updated by the `ANALYZE` command or during `autovacuum` operations.

--------

3. Viewing Query Planner Statistics
You can view the collected statistics using the pg_stats system catalog.

Example:


In [ ]:
SELECT * FROM pg_stats WHERE tablename = 'employees';

----

4. `Updating Statistics`
- To ensure the query planner has accurate statistics, you can manually run the `ANALYZE` command:



In [ ]:
ANALYZE employees;

-- This updates the statistics for the employees table.


-- To analyze all tables in the database:
ANALYZE;

-------------

5) `Example: Impact of Statistics on Query Planning`

- Without Accurate Statistics:
    - If statistics are outdated, the query planner might choose a sequential scan instead of an index scan, even when an index exists.

In [ ]:
EXPLAIN SELECT * FROM employees WHERE salary > 50000;


-- output:
Seq Scan on employees  (cost=0.00..1000.00 rows=100 width=50)

After Updating Statistics:
Run `ANALYZE` to update statistics:



In [ ]:
ANALYZE employees;

In [ ]:
EXPLAIN SELECT * FROM employees WHERE salary > 50000;

-- output:
Index Scan using idx_salary on employees  (cost=0.00..50.00 rows=100 width=50)


-- Now, the query planner uses the index scan instead of a sequential scan, which improves performance.

-----------

6. Configuring Statistics
- You can adjust the level of detail collected for a column using the ALTER TABLE command with the SET STATISTICS option.

Example:

In [ ]:
ALTER TABLE employees ALTER COLUMN salary SET STATISTICS 100;

-- The default statistics target is 100.
-- Higher values (up to 1000) collect more detailed statistics but increase the cost of ANALYZE.

---------------

7. Best Practices
- Regularly run ANALYZE or ensure autovacuum is enabled to keep statistics up-to-date.

- Use EXPLAIN or EXPLAIN ANALYZE to verify that the query planner is making optimal decisions.
- Adjust the statistics target for columns with skewed or non-uniform data distributions.


---------
-----------


Re-run the query:

#### 2. Types of Vacuuming
- a) `Manual Vacuum`
    - You can manually run the `VACUUM` command to clean up dead tuples.

In [ ]:
VACUUM employees;

-- This reclaims space but does not lock the table for reads or writes.

-- It does not update query planner statistics.

-  b) `Vacuum Full`

    - `VACUUM FULL` is a more aggressive version that:
    
    - Rewrites the entire table to remove dead tuples.
    - Locks the table, preventing reads and writes during the operation.

In [ ]:
VACUUM FULL employees;

-- Use this only when you need to reclaim a significant amount of space.

- `c) Autovacuum`

    - PostgreSQL has an autovacuum daemon that automatically runs VACUUM and ANALYZE in the background.

    - It is triggered based on thresholds (e.g., the number of dead tuples in a table).


#### Practical Examples

- a) Basic Vacuum

In [ ]:
VACUUM employees;
--  Run a simple vacuum to clean up dead tuples:

- b) Vacuum with Analyze

In [ ]:
VACUUM ANALYZE employees;
-- To clean up dead tuples and update query planner statistics:

- c) Vacuum Full

In [ ]:
VACUUM FULL employees;
-- To reclaim maximum space:
-- This locks the table, so it should be used during maintenance windows.

- d) Check Dead Tuples

In [ ]:
SELECT relname AS table_name, n_dead_tup AS dead_tuples
FROM pg_stat_user_tables
WHERE relname = 'employees';

-- You can check the number of dead tuples in a table using the pg_stat_user_tables view:

##### 5. Practical Use Case
- Scenario: Frequent Updates in a Table

- Suppose you have a table orders that is frequently updated or deleted.

In [ ]:
-- 1) Check Dead Tuples:

SELECT relname AS table_name, n_dead_tup AS dead_tuples
FROM pg_stat_user_tables
WHERE relname = 'orders';



-- 2) Run Vacuum:

VACUUM ANALYZE orders;

3) Enable Autovacuum for the Table: If autovacuum is not cleaning the table efficiently, you can adjust its settings for the orders table:

In [ ]:
ALTER TABLE orders SET (autovacuum_vacuum_threshold = 100, autovacuum_vacuum_scale_factor = 0.05);

In [ ]:
-- 4) Monitor Autovacuum:

SELECT relname, last_autovacuum, n_dead_tup
FROM pg_stat_user_tables
WHERE relname = 'orders';

6. Best Practices
- Enable Autovacuum: Always keep autovacuum enabled unless you have a specific reason to disable it.

- Monitor Dead Tuples: Regularly check for dead tuples in large or frequently updated tables.
- Schedule Vacuum Full: Use VACUUM FULL during maintenance windows for heavily bloated tables.
- Tune Autovacuum Settings: Adjust autovacuum thresholds for high-traffic tables.


By understanding and using vacuuming effectively, you can maintain optimal database performance and prevent storage bloat in PostgreSQL.

